# Homework 3

#### Members
Ron Cordell, Ted Dunmire, Filip Krunic

#### Emails (resp.)
glenn.dunmire.iv@gmail.com, *TedsEmail*, fkrunic@ischool.berkeley.edu

-----

## Question 3.0
**

#### Solution: 

Here's some text.

-----

## Question 3.1
**

#### Solution: 

Here's some text.

-----

## Question 3.2
**

#### Solution: 

Here's some text.

-----

## Question 3.3
*Using a single reducer: Report your findings such as:*
1. *Number of unique products*
2. *Largest basket*
3. *Report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. *

#### Solution: 

##### Mapper

Below is the mapper used to accomplish this task. It emits each token along with a basket size to the reducers for additional processing. 

In [73]:
%%writefile mapperQ33.py
#!/usr/bin/env python

import sys
from collections import defaultdict
from itertools import combinations

def readInput(file, separator=None):
	for line in file:
		yield line.split(separator)

if __name__ == "__main__":


	# Read input 
	data = readInput(sys.stdin)
	for line in data: 
		holdingDict = defaultdict(int)
		basketSize = len(line)


		# Append elements 
		for token in line: 
			holdingDict[token] += 1


		# Emit results 
		for k, v in holdingDict.iteritems(): 
			basketInfo = str([v, basketSize])
			print '%s%s%s' % (k, '\t', basketInfo)
            
	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Mappers,1\n")

Overwriting mapperQ33.py


##### Reducer

Below is the reducer used to accomplish this task. The reducer uses a `defaultdict` object from `collections` to automatically collate tokens and their counts without having to explicitly instantiate the key. This is convenient when reading from `sys.stdin` as one can yield lines and store the tokens simultaneously. 

In [74]:
%%writefile reducerQ33.py
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict
import ast 

def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)


if __name__ == "__main__":

	# Final store 
	storingDict = defaultdict(int)
	maxBasket = 0
	totalTerms = 0


	# Read data 
	data = readInput(sys.stdin)
	for line in data: 


		# Parse value 
		token = line[0]
		termCount, basketSize = ast.literal_eval(line[1])
		totalTerms += termCount


		# Store results 
		storingDict[token] += termCount
		maxBasket = max(maxBasket, basketSize)


	# Metrics
	numUniqueProducts = len(set(storingDict.keys()))
	largestBasket = maxBasket


	# Compute frequencies 
	for k, v in storingDict.iteritems():
		storingDict[k] = v

	
	# Find most frequent terms 
	mostFrequentTerms = [(k, v, round(v / totalTerms, 4)) for k, v in storingDict.iteritems()]
	mostFrequentTerms = sorted(mostFrequentTerms, 
								key = lambda x: x[1], 
								reverse = True)


	# Get results 
	print '\n' + '========== Number of Unique Products ==========' + '\n'
	print 'Answer: ' + str(numUniqueProducts) + '\n'
    
	print '\n' + '========= Largest Basket ==========' + '\n'
	print 'Answer: ' + str(largestBasket) + '\n'    

	print '========== Most Frequent Terms ==========' + '\n'
	template = "{0:20}|{1:20}|{2:20}"
	print template.format("ITEM", "FREQUENCY", "RELATIVE FREQUENCY")
	
	# Print terms 
	for termPair in mostFrequentTerms[:50]: 
		print template.format(*termPair)
        
	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Reducers,1\n")

Overwriting reducerQ33.py


##### Wrapper

Below is the bash script that's used to submit the Hadoop Streaming job. It maps user input to variables and specifies the options for the job. 

In [29]:
%%writefile wrapperQ33.sh
#!/bin/bash


# Initialize
RAW_DATA=$1
RAW_MAPPER=$2
RAW_REDUCER=$3


# Hadoop variables 
HDFS_DIR="/user/john/notebook"
HDFS_INPUT="$HDFS_DIR/input"
HDFS_OUTPUT="$HDFS_DIR/output"
HDFS_FILES="$HDFS_DIR/files"


# Local variables 
PROJECT_DIR="/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook"
DATA="$PROJECT_DIR/$RAW_DATA"
MAPPER="$PROJECT_DIR/$RAW_MAPPER"
REDUCER="$PROJECT_DIR/$RAW_REDUCER"


# NAIVE="$PROJECT_DIR/naiveBayes.py"
STREAMING_JAR="$PROJECT_DIR/hadoop-streaming-2.6.0.jar"


# Make directories and put file 
hdfs dfs -rm -r $HDFS_DIR
hdfs dfs -mkdir $HDFS_DIR $HDFS_INPUT $HDFS_FILES
hdfs dfs -put $DATA $HDFS_INPUT


# Execute
hadoop jar $STREAMING_JAR \
	-file "$MAPPER" -mapper "$MAPPER" \
	-file "$REDUCER" -reducer "$REDUCER" \
	-input $HDFS_INPUT \
	-output $HDFS_OUTPUT 


# Output results  
if [ $? -eq 0 ]; then 
	hdfs dfs -cat $HDFS_OUTPUT/part-00000
fi	

Overwriting wrapperQ33.sh


##### Submit Job

Below we submit the job. The wrapper takes as arguments the data, mapper, and reducer. Later versions of the wrapper also allow combiner arguments. 

In [1]:
!bash wrapperQ33.sh ProductPurchaseData.txt mapperQ33.py reducerQ33.py

16/02/02 08:21:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 08:21:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/john/notebook
16/02/02 08:21:21 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 08:21:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 08:21:27 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/02/02 08:21:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook/mapperQ33.py, /Users/john/Dropbox/MIDS/W261/Week3/Homework

-----

## Question 3.4
*Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.*

*List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. *

*Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.*

#### Solution: 

##### Mapper

Below is the mapper code for the pairs implementation. It generates tuple-combinations from the input lines, sorts them to get unique keys, then emits them to the reducer or optional combiner. 

In [10]:
%%writefile mapperQ34.py
#!/usr/bin/env python

from __future__ import division 
import sys
from collections import defaultdict
from itertools import combinations

def readInput(file, separator=None):
	for line in file:
		yield line.split(separator)

if __name__ == "__main__":

	# Read input 
	data = readInput(sys.stdin)
	totalBaskets = 0

	for line in data: 

		# Increment
		totalBaskets += 1

		# Get unique keys 
		pairs = list(combinations(line, 2))
		

		# Sort keys 
		sortedPairs = []
		for pair in pairs: 
			pList = list(pair)
			pList.sort()
			sortedPairs.append(tuple(pList))

		# Emit
		for pair in sortedPairs: 
			print '%s%s%s' % (pair, '\t', 1)
			
		
	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Mappers,1\n")

	# Emit basket count 
	print '%s%s%s' % ('*', '\t', str(totalBaskets))

Overwriting mapperQ34.py


##### Reducer

Below is the reducer for this process. It uses a simple key-aggregation to collate the results from the mappers, where the keys are unique tuples from the lines read in by the mapper. The uniqueness condition is specified at the line-level when read by the mapper. 

In [11]:
%%writefile reducerQ34.py
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict
import ast 


def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)


if __name__ == "__main__":

	# Final store 
	storingDict = defaultdict(int)
	support = 100
	totalBaskets = 0


	# Read data 
	data = readInput(sys.stdin)
	for line in data: 

		# Check for basket 
		token = line[0]

		if token == '*': 
			totalBaskets += int(line[1])

		else:

			# Parse 
			termCount = int(line[1])

			# Store results 
			storingDict[token] += termCount

	
	# Filter
	filterDict = defaultdict(int)
	for k, v in storingDict.iteritems(): 
		if v >= support:
			filterDict[k] += v 


	# Find most frequent terms 
	mostFrequentTerms = [(k, v, round(int(v) / totalBaskets, 3)) for k, v in filterDict.iteritems()]
	mostFrequentTerms = sorted(mostFrequentTerms, 
								key = lambda x: x[1], 
								reverse = True)


	# Get results 
	print '\n' + '========== Most Frequent Terms ==========' + '\n'
	template = "{0:30}|{1:20}|{2:20}"
	print template.format("PAIR", "SUPPORT COUNT", "RELATIVE FREQUENCY")
	
	# Print terms 
	for termPair in mostFrequentTerms[:50]: 
		print template.format(*termPair)
        
	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Reducers,1\n")

Overwriting reducerQ34.py


##### Combiner (Optional)

Below is the code for an optional combiner. It has the same signature as the reducer and does the same essential aggregation. 

In [12]:
%%writefile combinerQ34.py
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict
import ast 


def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)


if __name__ == "__main__":

	# Final store 
	storingDict = defaultdict(int)


	# Read data 
	data = readInput(sys.stdin)
	totalBaskets = 0

	
	for line in data: 

		# Check for basket 
		token = line[0]

		if token == '*': 
			totalBaskets += int(line[1])

		else:

			# Parse 
			termCount = int(line[1])

			# Store results 
			storingDict[token] += termCount


	# Emit
	for k, v in storingDict.iteritems(): 
		print '%s%s%s' % (k, '\t', v)
        
	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Combiners,1\n")

	# Emit basket count 
	print '%s%s%s' % ('*', '\t', str(totalBaskets))

Overwriting combinerQ34.py


##### Wrapper

Here is the wrapper for this particular submission. The noticeable difference is that it now has been modified to pass a combiner file to Hadoop Streaming. 

In [52]:
%%writefile wrapperQ34.sh
#!/bin/bash


# Initialize
RAW_DATA=$1
RAW_MAPPER=$2
RAW_REDUCER=$3
RAW_COMBINER=$4


# Hadoop variables 
HDFS_DIR="/user/john/notebook"
HDFS_INPUT="$HDFS_DIR/input"
HDFS_OUTPUT="$HDFS_DIR/output"
HDFS_FILES="$HDFS_DIR/files"


# Local variables 
PROJECT_DIR="/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook"
DATA="$PROJECT_DIR/$RAW_DATA"
MAPPER="$PROJECT_DIR/$RAW_MAPPER"
REDUCER="$PROJECT_DIR/$RAW_REDUCER"
COMBINER="$PROJECT_DIR/$RAW_COMBINER"


# NAIVE="$PROJECT_DIR/naiveBayes.py"
STREAMING_JAR="$PROJECT_DIR/hadoop-streaming-2.6.0.jar"


# Make directories and put file 
hdfs dfs -rm -r $HDFS_DIR
hdfs dfs -mkdir $HDFS_DIR $HDFS_INPUT $HDFS_FILES
hdfs dfs -put $DATA $HDFS_INPUT


# Execute
hadoop jar $STREAMING_JAR \
	-file "$MAPPER" -mapper "$MAPPER" \
	-file "$REDUCER" -reducer "$REDUCER" \
    -file "$COMBINER" -combiner "$COMBINER" \
	-input $HDFS_INPUT \
	-output $HDFS_OUTPUT \
	-cmdenv mapred.map.max.attempts=1 \
	-cmdenv mapred.reduce.max.attempts=1 \


# Output results  
if [ $? -eq 0 ]; then 
	hdfs dfs -cat $HDFS_OUTPUT/part-00000
fi	

Overwriting wrapperQ34.sh


##### Submit Job

We now submit the job for this process. Notice that an argument has been added for the combiner. 

In [13]:
!bash wrapperQ34.sh ProductPurchaseData.txt mapperQ34.py reducerQ34.py combinerQ34.py

16/02/01 19:56:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:56:53 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/john/notebook
16/02/01 19:56:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:56:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:57:01 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/02/01 19:57:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook/mapperQ34.py, /Users/john/Dropbox/MIDS/W261/Week3/Homework

To view the counters, check the 'User-Defined' category in the job logs appearing just prior to the reducer results. It should appear as follows: 

	User-Defined
		Number of Combiners=2
		Number of Mappers=1
		Number of Reducers=1

-----

## Question 3.5
*Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.*

*Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)*

*Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs*

#### Solution: 

##### Mapper

The mapper for the stripes implementation is noticeably different than the one used for the pairs implementation. In particular, a dictionary stripe is emitted which is then parsed literally by the reducer or optional combiner. From here, the stripes are aggregated per token then divided by two to compensate for the two combinations in which a key can be updated. 

In [1]:
%%writefile mapperQ35.py
#!/usr/bin/env python

import sys
from collections import defaultdict
from itertools import combinations

def readInput(file, separator=None):
	for line in file:
		yield line.split(separator)

if __name__ == "__main__":

	# Read input 
	data = readInput(sys.stdin)
	totalBaskets = 0
    
	for line in data: 
		
		totalBaskets += 1
		
		for token in line: 

			occurrence = defaultdict(int) 

			# Remove token from neighbors
			stripe = [x for x in line if x != token]
			
			# Create co-occurrence array 
			for neighbor in stripe: 
				occurrence[neighbor] += 1

			# Emit
			cArray = dict(occurrence)

			print '%s%s%s' % (token, '\t', str(cArray))

	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Mappers,1\n")

	# Emit basket count
	print '%s%s%s' % ('*', '\t', str(totalBaskets))

Overwriting mapperQ35.py


##### Reducer

Here, the reducer has been changed to compensate for the adjusted input. The reducer aggregates the stripes by aggregating the incoming dictionaries. It does this efficiently using `defaultdict`, an automatically instantiating key-value dictionary included in `collections`. 

In [8]:
%%writefile reducerQ35.py 
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict
from itertools import combinations, chain 
import ast 


# Read input from mapper 
def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)	


if __name__ == "__main__":

	tokenDict = defaultdict(list)
	mergedDict = defaultdict(dict)
	tupleList = defaultdict(float)
	flattenedTerms = defaultdict(int)
	support = 100
	totalBaskets = 0


	# Read data 
	data = readInput(sys.stdin)
	for line in data: 

		
		token = line[0]
		if token == '*': 

			# Increment basket 
			totalBaskets += int(line[1])	

		else:

			# Parse normally 
			stripe = ast.literal_eval(line[1])
				
			# Combine dictionaries 
			tokenDict[token].append(stripe) 


	# Merge stripes 
	for k, v in tokenDict.iteritems(): 
		merged = defaultdict(int)
		
		# Loop and aggregate 
		for stripe in v: 
			for stripeKey in stripe: 
				merged[stripeKey] += stripe[stripeKey]

		mergedDict[k] = merged


	# Create key-value pairs 
	for token, stripe in mergedDict.iteritems(): 
		for innerToken, count in stripe.iteritems(): 

			# Get unique keys 
			tokenPair = [token, innerToken]
			tokenPair.sort()
			tuplePair = tuple(tokenPair)

			# Overcounting exactly twice per pair
			tupleList[tuplePair] += count / 2
	


	# Find most frequent terms and filter
	mostFrequentTerms = [(k, int(v), round(int(v) / totalBaskets, 3)) for k, v in tupleList.iteritems() 
                         if int(v) >= support]
    
	mostFrequentTerms = sorted(mostFrequentTerms, 
								key = lambda x: x[1], 
								reverse = True)


	# Get results 
	print '\n' + '========== Most Frequent Terms ==========' + '\n'
	template = "{0:30}|{1:20}|{2:20}"
	print template.format("PAIR", "SUPPORT COUNT", "RELATIVE FREQUENCY")
	
	# Print terms 
	for termPair in mostFrequentTerms[:50]: 
		print template.format(*termPair)

	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Reducers,1\n")        

Overwriting reducerQ35.py


##### Combiner

Below is the combiner used optionally in the process. It aggregates in a similar way to the reduce and outputes partially aggregated stripes. 

In [3]:
%%writefile combinerQ35.py
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict 
import ast 


# Read input from mapper 
def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)

if __name__ == "__main__":

	tokenDict = defaultdict(list)
	mergedDict = defaultdict(dict)
	totalBaskets = 0


	# Read data 
	data = readInput(sys.stdin)
	for line in data: 

		token = line[0]
		if token == '*': 

			# Increment basket 
			totalBaskets += int(line[1])	

		else:

			# Parse normally 
			stripe = ast.literal_eval(line[1])
				
			# Combine dictionaries 
			tokenDict[token].append(stripe) 


	# # Merge stripes 
	for k, v in tokenDict.iteritems(): 
		merged = defaultdict(int)
		
		# Loop and aggregate 
		for stripe in v: 
			for stripeKey in stripe: 
				merged[stripeKey] += stripe[stripeKey]

		mergedDict[k] = dict(merged)

	# Emit results 
	for k, v in mergedDict.iteritems(): 
		print '%s%s%s' % (k, '\t', str(v))

	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Combiners,1\n")      

	# Emit baskets 
	print '%s%s%s' % ('*', '\t', str(totalBaskets))          

Overwriting combinerQ35.py


##### Submit Job 

Our wrapper remains unchanged from the prior implementation that has an argument input for the combiner. We re-use it here:

In [9]:
!bash wrapperQ34.sh ProductPurchaseData.txt mapperQ35.py reducerQ35.py combinerQ35.py

16/02/01 19:50:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:50:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/john/notebook
16/02/01 19:50:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:50:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:50:42 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/02/01 19:50:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook/mapperQ35.py, /Users/john/Dropbox/MIDS/W261/Week3/Homework

The job was run on a Macbook Pro with a 2.66GHz Intel dual-core processor and 4GB of memory. 

Based on the results from Question 3.4 and Question 3.5, the two jobs use the same number of combiners, mappers, and reducers to accomplish their respective tasks. These are 2 combiners, 1 mapper, and 1 reducer for both jobs. 

-----

## Question 3.6 (Optional)
### Part B

*What is the Apriori algorithm? Describe an example use in your domain of expertise.*

#### Solution: 

The Apriori algorithm is an algorithm for associative rule learning over transactional data sets. In particular, it implements a multi-scan approach with different tuning parameters like minimum support and minimum confidence. These are used to improve efficiency and control performance. In particular, the Apriori algorithm will identify frequent terms from the given support, and recurisvely compute larger baskets and search for additional frequent items. The Apriori algorithm stops when no further baskets can be found that are frequent. 

Within the domain of telecommunications, Apriori-like algorithms are used to describe the customer journey from signing-up to porting out. Identifying patterns that lead to churn is very important for Big Telecom, as the market continues to grow more volatile and competitive. Looking at information like clicks, calls to customer support, or billing events and building associative rules is important in finding business insights that improve customer retention. 


### Part B
*Define confidence and lift*

#### Solution: 

Given elements A and B, the confidence of A => B is equal to the support of A and B divided by the support of A. In words, confidence represents the likelihood of observing a basket containing B given a basket containing A. 

Lift is defined as a measure of performance of a model at predicting enhanced responses against a random choice targetting. In other words, Lift describes how well a model performs in identifying association rules that are more confident than selecting association rules at random. Given elements A and B, the lift of A => B is defined as the confidence of A and B given the average confidence across the entire basket list. 

-----

## Question 3.7 (Optional)
*Write a program using the A-priori algorithm
to find products which are frequently browsed together. Fix the support to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find itemsets of size 2 and 3.*

#### Solution: 

##### Mapper

We use the pairs implementation here as it requires minimal code changes, albeit being less efficient. To this end, the mapper now outputs both 2 and 3-tuples which are treated the same by the reducer. The reducer changes its printing functionality to this end.

In [2]:
%%writefile mapperQ37.py
#!/usr/bin/env python

from __future__ import division 
import sys
from collections import defaultdict
from itertools import combinations

def readInput(file, separator=None):
	for line in file:
		yield line.split(separator)

if __name__ == "__main__":

	# Read input 
	data = readInput(sys.stdin)
	totalBaskets = 0
	
	for line in data: 

		totalBaskets += 1

		# Get unique keys 
		doublePairs = list(combinations(line, 2))
		triplePairs = list(combinations(line, 3))
		

		# Sort doubles 
		sortedDoubles = []
		for pair in doublePairs: 
			pList = list(pair)
			pList.sort()
			sortedDoubles.append(tuple(pList))

		# Emit
		for pair in sortedDoubles: 
			print '%s%s%s' % (pair, '\t', 1)


		# Sort triples 
		sortedTriples = []
		for pair in triplePairs: 
			pList = list(pair)
			pList.sort()
			sortedTriples.append(tuple(pList))

		# Emit
		for pair in sortedTriples: 
			print '%s%s%s' % (pair, '\t', 1)

	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Mappers,1\n")

	# Emit basket count
	print '%s%s%s' % ('*', '\t', str(totalBaskets))

Overwriting mapperQ37.py


##### Reducer

We modify the reducer so it outputs the top 50 of both the length-2 and length-3 tuples. This is mainly a cosmetic change in the printing functionality. 

In [5]:
%%writefile reducerQ37.py
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict
import ast 


def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)


if __name__ == "__main__":

	# Final store 
	storingDict = defaultdict(int)
	support = 100
	totalBaskets = 0


	# Read data 
	data = readInput(sys.stdin)
	for line in data: 

		# Check for basket 
		token = line[0]

		if token == '*': 
			totalBaskets += int(line[1])

		else:

			# Parse 
			termCount = int(line[1])

			# Store results 
			storingDict[token] += termCount

	
	# Filter
	filterDict = defaultdict(int)
	for k, v in storingDict.iteritems(): 
		if v >= support:
			filterDict[k] += v 


	# Find most frequent doubles 
	mfqDoubles = [(k, v, round(int(v) / totalBaskets, 3)) for k, v in filterDict.iteritems()
					if k.count(',') == 1]

	mfqDoubles = sorted(mfqDoubles, 
								key = lambda x: x[1], 
								reverse = True)


	# Get results 
	print '\n' + '========== Most Frequent Doubles ==========' + '\n'
	template = "{0:30}|{1:20}|{2:20}"
	print template.format("PAIR", "SUPPORT COUNT", "RELATIVE FREQUENCY")
	
	# Print terms 
	for termPair in mfqDoubles[:50]: 
		print template.format(*termPair)



	# Find most frequent triples 
	mfqTriples = [(k, v, round(int(v) / totalBaskets, 3)) for k, v in filterDict.iteritems()
					if k.count(',') == 2]
					
	mfqTriples = sorted(mfqTriples, 
								key = lambda x: x[1], 
								reverse = True)


	# Get results 
	print '\n' + '========== Most Frequent Triples ==========' + '\n'
	template = "{0:50}|{1:20}|{2:20}"
	print template.format("PAIR", "SUPPORT COUNT", "RELATIVE FREQUENCY")
	
	# Print terms 
	for termPair in mfqTriples[:50]: 
		print template.format(*termPair)


Overwriting reducerQ37.py


##### Submit Job 

We now submit the job using the combiner and wrapper defined in Question 3.4. 

In [6]:
!bash wrapperQ34.sh ProductPurchaseData.txt mapperQ37.py reducerQ37.py combinerQ34.py

16/02/02 08:48:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 08:48:18 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/john/notebook
16/02/02 08:48:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 08:48:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 08:48:26 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/02/02 08:48:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook/mapperQ37.py, /Users/john/Dropbox/MIDS/W261/Week3/Homework

-----

## Question 3.8 (Optional)
**

#### Solution: 

Here's some text.